# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
city_df = pd.read_csv("./City_data.csv")
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,norilsk,69.3535,88.2027,15.42,78,57,5.73,RU,1651285707
1,punta arenas,-53.1500,-70.9167,30.31,93,0,5.75,CL,1651285696
2,avarua,-21.2078,-159.7750,82.45,78,95,17.27,CK,1651285708
3,hobart,-42.8794,147.3294,60.66,55,75,14.97,AU,1651285709
4,areosa,41.7178,-8.8571,58.30,71,19,3.89,PT,1651285709


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70) \
                                    & (city_df["Wind Speed"] < 10) \
                                    & (city_df["Cloudiness"] == 0)].dropna()
narrowed_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
193,sao miguel do araguaia,-13.2750,-50.1628,71.80,60,0,2.84,BR,1651285801
255,saint-pierre,-21.3393,55.4781,74.88,74,0,6.91,RE,1651285831
291,natal,-5.7950,-35.2094,75.42,94,0,5.75,BR,1651285849
345,vila velha,-20.3297,-40.2925,72.95,93,0,2.30,BR,1651285875
378,laguna,38.4210,-121.4238,78.78,12,0,8.05,US,1651285890
443,hope,33.6671,-93.5916,75.25,61,0,9.98,US,1651285920


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_df = narrowed_df
hotel_df['Hotel Name'] = ""

for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
        # change location each iteration while leaving original params in place
        params = {"keyword": "hotel","radius": 5000,"key": g_key}

        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df.head()        

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
193,sao miguel do araguaia,-13.2750,-50.1628,71.80,60,0,2.84,BR,1651285801,Hotel São Miguel
255,saint-pierre,-21.3393,55.4781,74.88,74,0,6.91,RE,1651285831,Hôtel Le Saint-Pierre
291,natal,-5.7950,-35.2094,75.42,94,0,5.75,BR,1651285849,Yak Beach Hotel
345,vila velha,-20.3297,-40.2925,72.95,93,0,2.30,BR,1651285875,Hotel Santorini
378,laguna,38.4210,-121.4238,78.78,12,0,8.05,US,1651285890,Hampton Inn & Suites Sacramento-Elk Grove Lagu...


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
fig = gmaps.figure()
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, 
                                 info_box_content=hotel_info)

# Display figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))